In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from ipywidgets import *
from matplotlib.animation import FuncAnimation
import matplotlib.animation as ani
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.tools import *
from cavities import *

## Paraxial modes and degeneracy

In [3]:
cavfct = SixMirror
parname = 'dx'
waists_vs_param(cavfct, parname, 1e-2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

27.769071237458192

In [4]:
res = degeneracy_length(cavfct, parname, 1e-3)
Ldeg = res.x
print(res)

     fun: 0.2200334382795486
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 27.76829940260428


## Paraxial eigenrays and hit pattern

In [27]:
elements = cavfct(**{parname: Ldeg})
sys = RaySystem(elements)
system = sys.abcd
mu1, mu2 = system.q

In [ ]:
rs = np.linspace(0., 0.5, 500)
distsin = np.zeros_like(rs)
dists = np.zeros_like(rs)
distsmax = np.zeros_like(rs)
ap = 0.
for i, r in enumerate(rs):
    mu = np.real(r*np.exp(1j*ap)*mu1)
    distsin[i] = np.linalg.norm(mu[:2])
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
    hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
    dists[i] = np.linalg.norm(hit_scr[-1,:])
    distsmax[i] = np.max(np.linalg.norm(hit_scr, axis=1))
    
plt.figure()
plt.plot(distsin, dists)
plt.plot(distsin, distsmax)
plt.xlabel(r'$r_{in}$ [mm]')
plt.ylabel(r'$r_{out}$ [mm]')
plt.title(r'$\mu = Re(a_r \mu_1)$')
plt.show()

In [5]:
def make_divergence_plot(cavfct, parname, ax=None, dl=0.):
    res = degeneracy_length(cavfct, parname, 1e-3)
    print(res)
    Ldeg = res.x

    elements = cavfct(**{parname: Ldeg+dl})
    sys = RaySystem(elements)
    system = sys.abcd
    mu1, mu2 = system.q
    
    #scale in waists
    waist = system.waist_at(0)[0]
    rmu = np.linalg.norm(np.real(mu1[:2]))
    
    rs = np.linspace(0., 10., 300)
    distsin = np.zeros_like(rs)
    dists = np.zeros_like(rs)
    distsmax = np.zeros_like(rs)
    ap = 0.
    for i, r in enumerate(rs):
        mu = np.real(r*waist/rmu*mu1)
        distsin[i] = np.linalg.norm(mu[:2])
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        dists[i] = np.linalg.norm(hit_scr[-1,:])
        distsmax[i] = np.max(np.linalg.norm(hit_scr, axis=1))
    
    if ax is None:
        f, ax = plt.subplots()
    ax.plot(distsin, dists-distsin)
    ax.plot(distsin, distsmax-distsin)
    ax.set_xlabel(r'$r_{in}$ [mm]')
    ax.set_ylabel(r'$r_{out}$ [mm]')
    ax.set_title(r'$\mu = Re(a_r \mu_1)$')
    ax.axvline(waist, color='grey')
    if ax is None:
        return f

In [6]:
make_divergence_plot(SixMirror, 'dx', dl=-0.0)

     fun: 0.2200334382795486
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 27.76829940260428


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
make_divergence_plot(OriginalTwister, 'betal')

     fun: 0.8964813996871679
 message: 'Solution found.'
    nfev: 7
  status: 0
 success: True
       x: 31.675317130699597


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
waists_vs_param(GravEM, 'l', 1e-2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

51.00170568561873

In [8]:
make_divergence_plot(GravEM, 'l')

     fun: 0.00010495407150133088
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 51.000858373814026


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
waists_vs_param(LensCav, 'lens_dist', 1e-2)
res = degeneracy_length(LensCav, 'lens_dist', 1e-2)
Ldeg = res.x
print(res)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

     fun: 0.2747336422630812
 message: 'Solution found.'
    nfev: 8
  status: 0
 success: True
       x: 21.524499087168326


In [57]:
waists_vs_param(Lens_cav, 'lens_dist', 1e-2)

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\tools.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  g, ax = plt.subplots(ncols=2, figsize=(8,4))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

25.11594983277592

In [23]:
make_divergence_plot(LensCav, 'lens_dist')

     fun: 0.015683535957800906
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 21.52449773968428


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
elements = cavfct(**{parname: Ldeg})
sys = RaySystem(elements)
system = sys.abcd
mu1, mu2 = system.q

rs = np.linspace(0., 0.1, 100)
ps = np.linspace(0., 2*np.pi, 100)
dists = np.zeros((rs.shape[0], ps.shape[0]))
for i, r in enumerate(rs):
    for j, p in enumerate(ps):
        mu = np.real(r*np.exp(1j*p)*mu1)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        #dists[i,j] = np.linalg.norm(hit_scr[-1,:])
        dists[i,j] = np.max(np.linalg.norm(hit_scr, axis=1))

In [14]:
plt.figure()
plt.pcolormesh(rs, ps, dists.T)

plt.xlabel(r'$r$')
plt.ylabel(r'$\phi$')
plt.title(r'$\mu = Re(r \mu_1 e^{{i\phi}})$')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Main method comparison

In [24]:
fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
ms = 8.
linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
ax.set_title('Raytracing')
bx.set_title('ABCD')
cx.set_title('ABCD (Extracted)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
 
cavfct = LensCav
parname = 'lens_dist'
res = degeneracy_length(cavfct, parname, 1e-3)
Ldeg = res.x
print(res)

def makeidx(hit_m):
    return np.arange(hit_m.shape[0])

def update(dl=0., ar=0.0001, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        elements = cavfct(**{parname: Ldeg+dl*1e-2})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q

        mu = np.real(ar*np.exp(1j*ap)*mu1 + br*np.exp(1j*bp)*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        abcd_fd = sys.extract_ABCD(epsr=1e-3, epss=1e-3, Nrt=1) 
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.relim(visible_only=True)
        ax.autoscale_view(tight=True)
        fig.canvas.draw_idle()
        
interactive(update, dl=(-1, 1, 1e-3), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,2000,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

     fun: 0.015683535957800906
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 21.52449773968428


interactive(children=(FloatSlider(value=0.0, description='dl', max=1.0, min=-1.0, step=0.001), FloatSlider(val…

In [ ]:
plt.close('all')